# Python Assessment: Monthly Bill Generator (Option-1)



## Generates a bill by calculating charges for active items and grouping them appropriately.

### Requirements

##### 1. Active Item Detection
- Include only items whose date range intersects with the selected month.

##### 2.Grouping Logic
-  Item_code and rate matchs and their billing periods/intersection of item's start and stop dates with selected month.


##### 3. Amount Calculation
- Calculate the billable amount for each item based on its active days in the selected month.


##### 4. Total Revenue
- The total revenue should be the sum of all grouped amounts.

##  Final code

###### Press ctrl+Enter to get output

In [ ]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from collections import defaultdict

def generate_monthly_bill(item_list: list, target_month: str):
    # Parse target month
    year, month = map(int, target_month.split('-'))
    month_start = datetime(year, month, 1)
    month_end = month_start + relativedelta(months=1) - timedelta(days=1)
    
    # Process items here
    processed_items = []
    for item in item_list:
        try:
            processed = {
                'item_code': item['item_code'],
                'qty': float(item['qty']) if isinstance(item['qty'], str) else item['qty'],
                'rate': float(item['rate']) if isinstance(item['rate'], str) else item['rate'],
                'start_date': datetime.strptime(item['start_date'], '%Y-%m-%d'),
                'stop_date': datetime.strptime(item['stop_date'], '%Y-%m-%d') if item.get('stop_date') else None
            }
            processed_items.append(processed)
        except (KeyError, ValueError):
            continue
    
    # Filter active items
    active_items = []
    for item in processed_items:
        if (item['start_date'] <= month_end) and (item['stop_date'] is None or item['stop_date'] >= month_start):
            active_items.append(item)
    
    # Grouping items
    groups = defaultdict(list)
    for item in active_items:
        start = max(item['start_date'], month_start)
        stop = min(item['stop_date'], month_end) if item['stop_date'] else month_end
        group_key = (item['item_code'], item['rate'], start.strftime('%Y-%m-%d'), stop.strftime('%Y-%m-%d'))
        groups[group_key].append(item)
    
    # Prepare output for above code
    line_items = []
    total_revenue = 0.0
    
    for key, items in groups.items():
        item_code, rate, start_str, stop_str = key
        start = datetime.strptime(start_str, '%Y-%m-%d')
        stop = datetime.strptime(stop_str, '%Y-%m-%d')
        
        total_qty = sum(item['qty'] for item in items)
        days_in_period = (stop - start).days + 1
        days_in_month = (month_end - month_start).days + 1
        
        amount = total_qty * rate * (days_in_period / days_in_month) if days_in_period < days_in_month else total_qty * rate
        
        line_items.append({
            'item_code': item_code,
            'rate': rate,
            'qty': total_qty,
            'amount': round(amount, 2),
            'billing_period': f"{start_str} to {stop_str}"
        })
        total_revenue += amount
    
    return {
        'line_items': line_items,
        'total_revenue': round(total_revenue, 2)
    }

# Test data (simplified)
item_list = [
    {
        "item_code": "Executive Desk (4*2)",
        "qty": 10,
        "rate": "1080",
        "start_date": "2024-11-01",
        "stop_date": "2024-11-30",
    }
]

# Test the function
bill = generate_monthly_bill(item_list, "2024-11")
print(bill)


# Python Assessment: Monthly Bill PDF Invoice Generator(Option-2)

## Purpose:
Generates a custom PDF invoice with customer details and itemized billing.

### Inputs

##### 1. Invoice title, customer name and email are collected from  user input.

##### 2.Item entries like name, quantity, price and description added interactively.


##### 3.It generate a PDF as output.


##  Python code

###### Press ctrl+Enter to get output

In [ ]:
from reportlab.lib.pagesizes import letter

from reportlab.pdfgen import canvas

def generate_custom_pdf(title, customer_data, items):
    pdf_file = f"{title}.pdf"

    c = canvas.Canvas(pdf_file, pagesize=letter)

    c.setFont("Helvetica", 12)

    c.drawString(50, 750, title)

    c.drawString(50, 700, "Customer Information: ")

    y_offset = 680

    for key, value in customer_data.items():
        c.drawString(50, y_offset, f"{key}: {value}")
        y_offset -= 20

    c.drawString(50, y_offset - 20, "Items")
    c.drawString(50, y_offset - 40, "Item Name")
    c.drawString(200, y_offset - 40, "Quantity")
    c.drawString(300, y_offset - 40, "Price")
    c.drawString(400, y_offset - 40, "Description")

    y_offset -= 60

    total_amount = 0

    for item in items:
        item_name, quantity, price, description = item

        c.drawString(50, y_offset, item_name)
        c.drawString(200, y_offset, str(quantity))
        c.drawString(300, y_offset, f"Rs.{price:.2f}")
        c.drawString(400, y_offset, description)

        total_amount += quantity * price
        y_offset -= 20

        c.drawString(200, y_offset - 20, "Total")

        c.drawString(300, y_offset - 20, f"Rs.{total_amount:.2f}")


        c.save()

if __name__ == "__main__":
    title = input("Enter the invoice title: ")

    customer_data = {}

    customer_data["Name"] = input("Enter Customer Name: ")

    

    customer_data["Email"] = input("Enter Customer Email: ")

    items = []

    while True:
        item_name = input("Enter Item Name (or 'done' ) to finish: ")

        if item_name.lower() == "done":
            break

        quantity = int(input("Enter Quantity: "))

        price = float(input("Enter Price: "))

        description = input("Enter Product Description: ")

        items.append((item_name, quantity, price, description))

        generate_custom_pdf(title, customer_data, items)

